#  Coevolução no jogo Rastros
## Projeto nº 3
### Introdução à Inteligência Artificial - edição 2020/21


## Grupo: 42

### Elementos do Grupo

Nome: Ivo Veiga

Número: 44865

Nome: João Silva

Número: 48782

Nome: Manuel Tovar

Número: 49522

## Relatório

### Funções características do jogo Rastros
As Funções características são a distância à base e distância ao objectivo.

A distância à base é a distância da peça branca à casa (4,5) e a distância ao objetivo é a distância da peça branca à casa (8,1) caso o jogador seja 'S' ou à casa (1,8) caso o jogador seja 'N'.

In [1]:
from rastros import *
import random

def dist(a, b):
    return max(abs(a[0]-b[0]), abs(a[1]-b[1]))

def dist_base(state,player):
    return dist(state.white, (4,5))

def dist_obj(state,player):
    goal = (8, 1)
    if player == "N":
        goal = (1, 8)
    return dist(state.white, goal)

#lista das funções características
list_fun_caracts = [dist_base, dist_obj]

### Função de avaliação, jogadores com cromossomas distintos e execução de pares de jogos

A associação de um jogador a um cromossoma é feita antes da execução do primeiro jogo de um par usando um dicionário em que a chave são os jogadores obtidos pelos atributos `first` e `second` de uma subclasse de Game e os valores são um par (instância de um jogo, cromossoma) correspondentes. No segundo jogo do par de jogos os valores são trocados. O dicionário (`p__game_chromossome` na implementação), bem como `list_fun_caracts` (funções características) são globais neste notebook para facilitar a realização do relatório passo a passo. No ficheiro IIA2021-proj3-42.py são variáveis de instância de classe, onde todo o algorimto coevol está encapsulado.

Assim, a função `utility` e o cromossoma que a função de avaliação vai usar dependem do jogador (`first`,`second`) que lhe for passado como argumento. Os jogadores são assim o par (instância de um jogo, cromossoma) associado a `first` e `second`, e não dependem de uma função de alfabeta e de avaliação específica a um jogador.

A função de avaliação tem o seguinte aspeto:

In [2]:
#dicionario jogador-jogo,cromossoma
p__game_chromossome = {}
    
def f_eval(state,player):
    game = p__game_chromossome[player][0]
    chromossome = p__game_chromossome[player][1]
    #estados terminais
    win_or_lose = game.utility(state, player)
    if win_or_lose == 1:
        return infinity
    elif win_or_lose == -1:
        return -infinity
    else:
        #combinacao linear dos genes peso * funcao caracteristica 
        res = 0
        for chrom_index, caract in enumerate(list_fun_caracts):
            res += chromossome[chrom_index] * caract(state,player)
        return res

A associação de um jogadores a dois cromossomas [2,4] e [1,3] para um primeiro jogo de um par é então:

In [3]:
game_a = Rastros()
p__game_chromossome[game_a.first] = (game_a, [2,4])
p__game_chromossome[game_a.second] = (game_a, [1,3])

E para o segundo jogo a ordem dos jogadores é invertida:

In [4]:
game_a = Rastros()
p__game_chromossome[game_a.first] = (game_a, [1,3])
p__game_chromossome[game_a.second] = (game_a, [2,4])

Como a função de avaliação usada pelo alfabeta é igual para ambos os jogadores, existe só uma função alfabeta genérica:

In [5]:
limite_prof_exemplo = 2
alfabeta_func_exemplo = lambda game, state: alphabeta_cutoff_search_new(state,game,limite_prof_exemplo,eval_fn=f_eval)

Para executar um par de jogos pode então ser definida a seguinte função:

In [6]:
def play_game_pair(game_class,chromossome1,chromossome2,alfabeta_func,verbose=False) :
    def play_one_game(game):
        estado = game.initial
        if verbose :
            game.display(estado)
        fim = False
        while not fim :
            jogada = alfabeta_func(game,estado)
            if jogada == None:
                return 1 if game.to_move(estado) == game.second else -1
            estado = game.result(estado,jogada)
            if verbose :
                game.display(estado)
            fim = game.terminal_test(estado) 
        return game.utility(estado, game.to_move(game.initial))
    #primeiro jogo
    game = game_class()
    p__game_chromossome[game.first] = (game, chromossome1)
    p__game_chromossome[game.second] = (game, chromossome2)
    res_with_chrom1_fst = play_one_game(game)    
    #segundo jogo
    game = game_class()
    p__game_chromossome[game.first] = (game, chromossome2)
    p__game_chromossome[game.second] = (game, chromossome1)        
    res_with_chrom2_fst = play_one_game(game)
    
    if res_with_chrom1_fst == res_with_chrom2_fst:
        return 1,1 #empate ambos têm 1 vitória
    elif res_with_chrom1_fst > res_with_chrom2_fst:
        return 2,0 #chrom1 tem 2 vitórias , chrom2 tem 0 vitórias
    else:
        return 0,2 #chrom1 tem 0 vitórias , chrom2 tem 2 vitórias    

Exemplo de execução de um par de jogos entre os cromossomas [3,4] e [2,3] com limite de profundidade 2 (limite_prof_exemplo definido anteriormente):

In [7]:
chrom1_wins, chrom2_wins = play_game_pair(Rastros, [3,4], [2,3], alfabeta_func_exemplo, verbose=False)
print("Vitórias do cromossoma 1: ", chrom1_wins)
print("Vitórias do cromossoma 2: ", chrom2_wins)

Vitórias do cromossoma 1:  2
Vitórias do cromossoma 2:  0


Para se executarem N pares de jogos entre dois cromossomas para cada limite de profundidade define-se a seguinte função:

In [8]:
def play_n_pairs_per_limit(game_class,chromossome1,chromossome2, num_pairs_per_limit, depth_limits):
    chrom1_res = 0
    chrom2_res = 0
    for depth in depth_limits:  
        alfabeta_func = lambda game, state: alphabeta_cutoff_search_new(state,game,depth,eval_fn=f_eval)        
        for i in range (num_pairs_per_limit):
            chrom1_wins, chrom2_wins = play_game_pair(game_class, chromossome1, chromossome2, alfabeta_func)
            chrom1_res += chrom1_wins
            chrom2_res += chrom2_wins
    return chrom1_res, chrom2_res

Resultados da execução de 20 pares de jogos entre dois cromossomas ([1,-100] e [3,2]) para cada limite da lista [1,2,3,4] de limites:

In [9]:
depth_limits = [1,2,3,4]
num_pairs = 20
chrom1 = [1,-100]
chrom2 = [3,2]
chrom1_wins, chrom2_wins = play_n_pairs_per_limit(Rastros, chrom1, chrom2, num_pairs, depth_limits)
print("Vitórias do cromossoma ", chrom1, " : ", chrom1_wins)
print("Vitórias do cromossoma ", chrom2, " : ", chrom2_wins)

Vitórias do cromossoma  [1, -100]  :  45
Vitórias do cromossoma  [3, 2]  :  115


Resultados da execução de mais 20 pares de jogos entre os dois cromossomas anteriores, agora com o cromossoma [3,2] como o cromossoma1 e [1,-100] como cromossoma2: 

In [10]:
chrom1 = [3,2]
chrom2 = [1,-100]
chrom1_wins, chrom2_wins = play_n_pairs_per_limit(Rastros, chrom1, chrom2, num_pairs, depth_limits)
print("Vitórias do cromossoma ", chrom1, " : ", chrom1_wins)
print("Vitórias do cromossoma ", chrom2, " : ", chrom2_wins)

Vitórias do cromossoma  [3, 2]  :  119
Vitórias do cromossoma  [1, -100]  :  41


Para comprovar que a função de avaliação e a execução dos jogos estão bem implementadas e integram corretamente os cromossomas é necessário que os resultados anteriores sejam consistentes, ou seja que se observe que o número de vitórias em relação ao total de jogos para um dado cromossoma seja aproximadamente igual quer ou não esse cromossoma seja o primeiro argumento da função de execução de jogos. Isto é o caso, já que o cromossoma [3,2] é consistentemente melhor que o [1,-100].

Uma implementação alternativa para funções de avaliação genérica para subclasses de Game sería o uso de uma classe FunAval com o um método que implementa a função de avaliação usada no alfabeta e em que no seu construtor são passados uma instância de Game, um cromossoma e a lista de funções características:

In [11]:
class FunAval():
    def __init__(self, game, chrom, fun_caracts):
        self.game = game
        self.chrom = chrom
        self.fun_caracts = fun_caracts
    def f_eval(self, state, player):
        #estados terminais
        win_or_lose = self.game.utility(state, player)
        if win_or_lose == 1:
            return 9999999999999999
        elif win_or_lose == -1:
            return -9999999999999999
        else:
            #combinacao linear dos genes peso * funcao caracteristica 
            res = 0
            for chrom_index, caract in enumerate(self.fun_caracts):
                res += self.chrom[chrom_index] * caract(state,player)
            return res

Para uma instância de Game (Rastros):

In [12]:
game_ex = Rastros()

Um jogador será então a função alfabeta definido do seguinte modo:

In [13]:
fun_aval_exemplo1 = FunAval(game_ex, [3,-1], [dist_base, dist_obj]).f_eval
jogador1 = lambda game, state: alphabeta_cutoff_search_new(state,game,2,eval_fn=fun_aval_exemplo1)

Um segundo jogador:

In [14]:
fun_aval_exemplo2 = FunAval(game_ex, [4,-2], [dist_base, dist_obj]).f_eval
jogador2 = lambda game, state: alphabeta_cutoff_search_new(state,game,2,eval_fn=fun_aval_exemplo2)

Desta forma os atributos `first` e `second` não são necessários.

Exemplo de um jogo:

In [15]:
does_first_player_win = game_ex.jogar(jogador2, jogador1, verbose=False)
print("jogador2 ganhou") if does_first_player_win == 1 else print("jogador1 ganhou")

jogador2 ganhou


Esta implementação não é usada pois para cada cromossoma e para cada novo jogo é criada uma nova instância de FunAval, o que uma significa pior performance do algoritmo coeval.

### População inicial
Cada cromossoma da população inicial terá genes representados por inteiros pré-determinados. Ou seja, a gene pool será previamente gerada.

Define-se então a função que cria a população inicial:

In [16]:
def init_population(dim, gene_pool, num_caracts):
    assert (dim & (dim-1) == 0) and dim != 0 #dim tem de ser potência de 2 (100 (4) and 011 (3) = 000)
    g = len(gene_pool)
    population = []
    for i in range(dim):
        new_individual = [gene_pool[random.randrange(0, g)] for c in range(num_caracts)]
        population.append(new_individual)
    return population

Usando o intervalo [-1000,1000] para os valores possíveis da gene pool, a população inicial pode ser criada da seguinte maneira:

In [17]:
gene_pool = [i for i in range(-1000,1001,1)]
pop_size = 8
population = init_population(pop_size, gene_pool, len(list_fun_caracts))
print(population)

[[-186, 879], [302, -910], [973, 620], [429, -879], [-780, -6], [582, 61], [779, -407], [998, -311]]


### Fitness: competição tipo taça
É feita a competição tipo taça entre a população para obter uma tabela de cromossomas e respetivos fitness. Esta tabela será progressivamente construída ao longo da competição de modo a que o fitness de cada cromossoma seja igual ao número de rondas em que se mantiveram na competição. O vencedor terá fitness igual ao número total de rondas + 1.

Um exemplo de uma tabela para uma população de 8 elementos pode ser [([1, 2], 1), ([-1, -2], 1), ([-5, -3], 1), ([2, -1], 1), ([-6, -7], 2), ([-6, 9], 2), ([0, -3], 3), ([-3, -2], 4)] , onde o cromossoma [1,2] tem fitness 1 e o cromossoma [-3,-2] tem fitness 4.

Cada ronda da competição é executada recursivamente em que os vencedores de uma ronda (`winners`) são os que competem na seguinte (`players`) e os perdedores são adicionados à tabela (`fitness_list`). Define-se a função do seguinte modo:

In [18]:
def do_cup_competition(population, game_class, num_game_pairs, depth_limits):    
    fitness_list = []
    
    def do_round(players, i_round):
        winners = []
        for i in range(int(len(players)/2)):
            c1 = players[i*2]
            c2 = players[(i*2)+1]            
            wins1, wins2 = play_n_pairs_per_limit(game_class, c1, c2, num_game_pairs, depth_limits)
            if wins1 == wins2:
                if random.choice([True,False]):
                    winners.append(c1)
                    fitness_list.append((c2,i_round))
                else:
                    winners.append(c2)
                    fitness_list.append((c1,i_round))
            elif wins1 > wins2:
                winners.append(c1)
                fitness_list.append((c2,i_round))
            else:
                winners.append(c2)
                fitness_list.append((c1,i_round))
                
        if len(winners) > 1:
            do_round(winners,i_round+1)
        else:
            fitness_list.append((winners[0], i_round+1))
    
    random.shuffle(population) #modifica var. population, devolve None
    do_round(population, 1) #constroi a fitness_list
    
    return fitness_list

Exemplo de como fazer uma competição tipo taça entre `population` previamente gerada, no jogo Rastros e com 4 pares de jogos para cada um dos limites 2 e 3 por duelo:

In [19]:
pop_fitness = do_cup_competition(population, Rastros, 4, [2,3])
print(pop_fitness)

[([429, -879], 1), ([302, -910], 1), ([582, 61], 1), ([-186, 879], 1), ([779, -407], 2), ([973, 620], 2), ([-780, -6], 3), ([998, -311], 4)]


### Seleção por torneio de K
Na seleção por K torneio, K cromossomas são inicialmente escolhidos aleatoriamente de uma população e desses K o que tiver maior fitness é selecionado.

Na implementação da seleção por torneio de K que se segue, a população é a tabela de fitness e por defeito o K é 2. Para aferir que a função escolhe o cromossoma com maior fitness corretamente é dada a opção de passar o argumento `verbose`, que por defeito é False.

In [20]:
def do_tournament_selection(fitness_table, k_rivais=2, verbose=False):      
    highest_fitness = 0
    best_chromosome = None
    
    k_choices = random.sample(fitness_table, k_rivais)
    if verbose:
        print("tournament selected chromosomes: ", k_choices)
        
    for chrom_fit in k_choices:
        if chrom_fit[1] > highest_fitness:
            highest_fitness = chrom_fit[1]
            best_chromosome = chrom_fit[0]
    
    return best_chromosome

Exemplo de uso com a tabela de fitness sendo aquela obtida anteriormente (`pop_fitness`):

In [21]:
selected_chrom = do_tournament_selection(pop_fitness, verbose=True)
print("highest fitness chromosome: ", selected_chrom)

tournament selected chromosomes:  [([582, 61], 1), ([779, -407], 2)]
highest fitness chromosome:  [779, -407]


### Geração dos dois descendentes por cruzamento de 1 ponto de corte
A partir de dois cromossomas progenitores serão gerados dois descendentes de modo que a parte esquerda do cromossoma do progenitor 1 se combine com a parte direita do progenitor 2 e a parte direita do progenitor 1 se combine com a parte esquerda do do progenitor 2. O ponto que define uma parte é escolhido aleatoriamente de modo a que um progenitor passe sempre pelo menos um gene e é assumido que ambos os progenitores têm um número igual de genes (e logo que cada cromossoma da população terá sempre um número igual de genes).

A função que implementa este processo é definida do seguinte modo:

In [22]:
def generate_2_descendents(parent1, parent2):
    char_num = len(parent1)
    cut_point = random.randrange(1, char_num)
    f1 = parent1[:cut_point] + parent2[cut_point:]
    f2 = parent2[:cut_point] + parent1[cut_point:]
    return f1,f2

Uso da função assumindo como progenitores os cromossomas [2,3,4] e [5,6,7]:

In [23]:
desc1, desc2 = generate_2_descendents([2,3,4], [5,6,7])
print("descendentes: ",desc1,desc2)

descendentes:  [2, 3, 7] [5, 6, 4]


Para estar correto neste caso, os descendentes têm de ser ou [2, 3, 7] e [5, 6, 4] ou [2, 6, 7] e [5, 3, 4].

### Possibilidade de mutação (em todos os genes)
Cada gene do novo cromossoma terá uma certa chance (`p_mut`) de ser alterado somando um valor inteiro escolhido aleatoriamente do intervalo [-`mut_range`,`mut_range`]. `mut_range` tem de ser um inteiro positivo e por defeito é 10.

Este comportamente está definido na seguinte função:

In [24]:
def mutate_all_genes(chromossome, p_mut, mut_range=10):       
    new_chrom = []
    for c in chromossome:
        if random.uniform(0, 1) <= p_mut: 
            new_carac = c + random.randrange(-mut_range, mut_range+1)
            new_chrom.append(new_carac)
        else:
            new_chrom.append(c)
    return new_chrom

Exemplo da mutação do cromossoma [45,1,-10] com 25% chance de cada gene se alterar.

In [57]:
mutated = mutate_all_genes([45,1,-10], 0.25)
print(mutated)

[45, -4, -2]


### Formação da geração seguinte (e uso do elitismo)
Como é necessário que população seja uma potência de dois, aumentar a população traduz-se no seu aumento exponencial (duplica) após cada nova geração, e isso torna o algoritmo impraticável em tempo útil para um grande número de gerações. Assim sendo, a geração seguinte terá sempre o mesmo tamanho de população que a geração que a produz, ou seja o tamanho da população será sempre constante.

O primeiro passo na formação da nova geração é a aplicação de um mecanismo de elitismo à geração progenitora sendo que o número de elites (`elite_size`) será a percentagem `p_elit` do tamanho da população. Usando a tabela de fitness (`fitness_table`) resultante da competição tipo taça, selecionam-se os N=`elite_size` individuos com maior fitness para se juntarem à nova geração. Na implementação assume-se que `fitness_table` está ordenado por ordem de fitness crescente.

Para popular o restante da nova geração são gerados novos cromossomas. O processo de geração consiste em primeiro fazem-se duas seleções por torneio de K, uma por progenitor, de seguida os dois progenitores geram dois cromossomas descendentes, finalmente, aplica-se a função de mutação aos descendentes e adicionam-se ambos os resultados à nova geração. Este processo repete-se até que o tamanho da nova geração seja igual ao da geração progenitora.

Segue-se a implementação:

In [26]:
def generate_new_gen_with_elitism(fitness_table, p_elit, k_rivais, p_mut):
    new_gen = []
    pop_size = len(fitness_table)
    
    elite_size = round(pop_size * p_elit)
    non_elite_size = pop_size - elite_size
    #assume-se que fitness_table está ordenado por ordem de fitness crescente
    for i in range(pop_size-1, non_elite_size-1, -1):        
        new_gen.append(fitness_table[i][0])    
    
    for i in range(int(non_elite_size/2)):
        parent1 = do_tournament_selection(fitness_table, k_rivais)
        parent2 = do_tournament_selection(fitness_table, k_rivais)
        
        desc1, desc2 = generate_2_descendents(parent1, parent2)
        
        m_desc1 = mutate_all_genes(desc1, p_mut)
        m_desc2 = mutate_all_genes(desc2, p_mut)
        
        new_gen.append(m_desc1)
        new_gen.append(m_desc2)
        
    diff_gen_sizes = pop_size - len(new_gen)
    if diff_gen_sizes != 0: #falta +1 individuo para gerar, acontece se elite_size for ímpar
        parent1 = do_tournament_selection(fitness_table, k_rivais)
        parent2 = do_tournament_selection(fitness_table, k_rivais)        
        desc1, _ = generate_2_descendents(parent1, parent2)  
        m_desc1 = mutate_all_genes(desc1, p_mut)
        new_gen.append(m_desc1)
        
    return new_gen

Exemplo de geração de uma nova geração a partir da geração anterior (`pop_fitness`), com 20% de elitismo, seleção de K=3 e 10% chance de mutações:

In [60]:
new_gen = generate_new_gen_with_elitism(pop_fitness, 0.2, 3, 0.1)
print(new_gen)

[[998, -311], [-780, -6], [779, 620], [973, -407], [779, 614], [973, -407], [998, -6], [-780, -311]]


Neste caso, os dois primeiros cromossomas são as elites e se, usando os mesmos parâmetros, se se repetir o processo de geração, devem-se obter os mesmos dois individuos elites:

In [61]:
new_gen = generate_new_gen_with_elitism(pop_fitness, 0.2, 3, 0.1)
print(new_gen)

[[998, -311], [-780, -6], [-780, -311], [998, -6], [998, -6], [-780, -311], [779, -407], [779, -407]]


### Função principal `coevol`
Começa-se por gerar a população inicial e segue-se o processo de coevolução:
1. Determina-se o fitness da população usando a função `do_cup_competition`
2. Gera-se a nova geração usando a função `generate_new_gen_with_elitism`

Este processo repete-se tantas vezes quantas as gerações que forem pretendidas e na ultima geração determina-se o fitness da população evoluida (`do_cup_competition`) para aferir o melhor cromossoma.

Ao longo da coevolução é construída uma lista com os melhores cromossomas de cada geração e respetivos fitness. Essa lista é o que a função devolve. Visto que a população se mantém constante e o nível de fitness depende do tamanho da população (devido à competição tipo taça), o fitness não serve para comparar cromossomas de gerações diferentes. Os melhores cromossomas de cada geração terão por isso o mesmo nível numérico de fitness.

De notar que são usadas globais (`p_chromossome` e `list_fun_caracts`) para integrar a função de avaliação do alfabeta com os cromossomas neste notebook, o que torna mais fácil a elaboração passo a passo do relatório. Em IA2021-proj3-42.py o algoritmo na sua totalidade é encapsulado numa classe, para que não hajam globais. Um exemplo de como utilizar esta classe encontra-se mais à frente.

Segue-se então a função `coevol`:

In [67]:
def coevol(gen,dim,gene_pool,elit,p_mut,k_rivais,lim_profs,n_jogos,jogo,caracts):
    global list_fun_caracts #global neste notebook, em IA2021-proj3-42.py é um membro de classe
    list_fun_caracts = caracts
    best_chromosomes = []
    
    population = init_population(dim, gene_pool, len(caracts))
    
    for i in range(gen):
        pop_fitness = do_cup_competition(population, jogo, n_jogos, lim_profs)   
        best_chromosomes.append(pop_fitness[-1])
        
        population = generate_new_gen_with_elitism(pop_fitness, elit, k_rivais, p_mut)
        
    pop_fitness = do_cup_competition(population, jogo, n_jogos, lim_profs)   
    best_chromosomes.append(pop_fitness[-1])
    
    return best_chromosomes

### Execução do algoritmo de coevolução para o jogo Rastros
Execução do algoritmo de coevolução para o jogo Rastros com distância à base e distância ao objectivo como características e limites do alfabeta 1 e 2:

In [66]:
gen = 30
dim = 128
gene_pool = [i for i in range(-250,251,1)]
elit = 0.2
p_mut = 0.15
k_rivais = 2 #não dá para meter k_rivais=2 por defeito em coevol sem trocar a ordem dos argumentos (SyntaxError: non-default argument follows default argument)
lim_profs = [1,2]
n_jogos = 7
jogo = Rastros
caracts = [dist_base, dist_obj]

best_chromosomes = coevol(gen,dim,gene_pool,elit,p_mut,k_rivais,lim_profs,n_jogos,jogo,caracts)

for i, c in enumerate(best_chromosomes):
    print("Melhor da geração ", i, ": ", c[0], " com fitness ", c[1])

Melhor da geração  0 :  [107, 20]  com fitness  8
Melhor da geração  1 :  [-200, -31]  com fitness  8
Melhor da geração  2 :  [207, 131]  com fitness  8
Melhor da geração  3 :  [195, 163]  com fitness  8
Melhor da geração  4 :  [-210, -9]  com fitness  8
Melhor da geração  5 :  [-105, 6]  com fitness  8
Melhor da geração  6 :  [-231, 8]  com fitness  8
Melhor da geração  7 :  [77, 4]  com fitness  8
Melhor da geração  8 :  [-165, 166]  com fitness  8
Melhor da geração  9 :  [248, 110]  com fitness  8
Melhor da geração  10 :  [-181, 37]  com fitness  8
Melhor da geração  11 :  [-176, 42]  com fitness  8
Melhor da geração  12 :  [-165, -1]  com fitness  8
Melhor da geração  13 :  [244, 34]  com fitness  8
Melhor da geração  14 :  [-163, 28]  com fitness  8
Melhor da geração  15 :  [133, 28]  com fitness  8
Melhor da geração  16 :  [193, 7]  com fitness  8
Melhor da geração  17 :  [195, 42]  com fitness  8
Melhor da geração  18 :  [-114, 36]  com fitness  8
Melhor da geração  19 :  [195, 

Segue-se a execução do algoritmo, mantendo os parâmetros anteriores mas com uma função característica adicional (`general_eval_42`) usada no projeto 2.

A definição da função característica:

In [62]:
def general_eval_42(state, player):
    def moves_from_point_42(point, state):
        alladjacent = [(point[0]+a, point[1]+b) for a in [-1,0,1] for b in [-1,0,1]]
        return [p for p in alladjacent
                if p not in state.blacks and p !=state.white and p !=point and p in state.fullboard]
    goal = (8, 1)
    if player == "N":
        goal = (1, 8)
    move_score = 0
    num_moves = 0
    for move in state.moves():
        for sub_move in moves_from_point_42(move, state):
            s_score = dist(move, goal) - dist(sub_move,goal)
            move_score += s_score
            num_moves += 1
    if num_moves == 0:
        return 3
    return (move_score / num_moves)

A execução:

In [68]:
caracts = [dist_base, dist_obj, general_eval_42]

best_chromosomes = coevol(gen,dim,gene_pool,elit,p_mut,k_rivais,lim_profs,n_jogos,jogo,caracts)

for i, c in enumerate(best_chromosomes):
    print("Melhor da geração ", i, ": ", c[0], " com fitness ", c[1])

Melhor da geração  0 :  [-178, 222, -74]  com fitness  8
Melhor da geração  1 :  [229, 0, 85]  com fitness  8
Melhor da geração  2 :  [-234, -71, 171]  com fitness  8
Melhor da geração  3 :  [244, 0, 85]  com fitness  8
Melhor da geração  4 :  [244, 0, 85]  com fitness  8
Melhor da geração  5 :  [229, 0, 85]  com fitness  8
Melhor da geração  6 :  [229, 0, 85]  com fitness  8
Melhor da geração  7 :  [233, 0, 85]  com fitness  8
Melhor da geração  8 :  [229, 0, 94]  com fitness  8
Melhor da geração  9 :  [-131, -6, 85]  com fitness  8
Melhor da geração  10 :  [229, 0, -86]  com fitness  8
Melhor da geração  11 :  [226, 0, 236]  com fitness  8
Melhor da geração  12 :  [-142, -5, -74]  com fitness  8
Melhor da geração  13 :  [226, 0, 236]  com fitness  8
Melhor da geração  14 :  [229, 0, 125]  com fitness  8
Melhor da geração  15 :  [228, 0, 236]  com fitness  8
Melhor da geração  16 :  [175, 0, 125]  com fitness  8
Melhor da geração  17 :  [247, 0, -100]  com fitness  8
Melhor da geração

Para a execução do algoritmo com limites de profundidade do alfabeta 4 e 5, incluindo a função característica adicional (general_eval_42). São usados os seguintes parâmetros:

In [69]:
gen = 20
dim = 64
gene_pool = [i for i in range(-1000,1001,1)]
elit = 0.2
p_mut = 0.25
k_rivais = 2
lim_profs = [4,5]
n_jogos = 5
jogo = Rastros
caracts = [dist_base, dist_obj, general_eval_42]

Como para os limites 4 e 5 o tempo de execução é grande, escolheram-se parâmetros pouco custosos, de forma a reduzir o tempo para algumas horas. Foi escolhida uma taxa relativamente alta de 25% de mutação já que o tamanho da população não é muito grande e cada gene é modificado por um valor delta reduzido (delta -10 a 10 vs gene pool de -1000 a 1000).

A execução para os limites 4 e 5:

In [27]:
best_chromosomes = coevol(gen,dim,gene_pool,elit,p_mut,k_rivais,lim_profs,n_jogos,jogo,caracts)

for i, c in enumerate(best_chromosomes):
    print("Melhor da geração ", i, ": ", c[0], " com fitness ", c[1])

Melhor da geração  0 :  [931, 270, 447]  com fitness  6
Melhor da geração  1 :  [182, 699, -875]  com fitness  6
Melhor da geração  2 :  [-499, 786, 610]  com fitness  6
Melhor da geração  3 :  [-932, 709, -870]  com fitness  6
Melhor da geração  4 :  [-932, 709, -870]  com fitness  6
Melhor da geração  5 :  [-932, 700, 549]  com fitness  6
Melhor da geração  6 :  [-940, 708, 618]  com fitness  6
Melhor da geração  7 :  [-423, 264, 618]  com fitness  6
Melhor da geração  8 :  [-932, 700, 549]  com fitness  6
Melhor da geração  9 :  [-944, 795, 618]  com fitness  6
Melhor da geração  10 :  [-944, 795, 618]  com fitness  6
Melhor da geração  11 :  [-932, 707, 550]  com fitness  6
Melhor da geração  12 :  [-937, 795, 613]  com fitness  6
Melhor da geração  13 :  [-947, 713, 616]  com fitness  6
Melhor da geração  14 :  [-947, 718, 618]  com fitness  6
Melhor da geração  15 :  [-956, 711, 616]  com fitness  6
Melhor da geração  16 :  [-936, 714, 616]  com fitness  6
Melhor da geração  17 :

Exemplo do uso da classe Coevol incluída em IA2021-proj3-42.py que encapsula o algoritmo coevol:

In [88]:
#parâmetros
gen = 5
dim = 16
gene_pool = [i for i in range(-250,251,1)]
elit = 0.2
p_mut = 0.25
k_rivais = 2
lim_profs = [1,2]
n_jogos = 5
jogo = Rastros
caracts = [dist_base, dist_obj, general_eval_42]

#imports
import importlib
imported_p3 = importlib.import_module("IIA2021-proj3-42")
Coevol = imported_p3.Coevol

#o algoritmo em ação
best_chromosomes = Coevol().coevol(gen,dim,gene_pool,elit,p_mut,k_rivais,lim_profs,n_jogos,jogo,caracts)

for i, c in enumerate(best_chromosomes):
    print("Melhor da geração ", i, ": ", c[0], " com fitness ", c[1])

Melhor da geração  0 :  [157, 30, 14]  com fitness  5
Melhor da geração  1 :  [-16, -62, 63]  com fitness  5
Melhor da geração  2 :  [-108, -76, 221]  com fitness  5
Melhor da geração  3 :  [-108, -76, 221]  com fitness  5
Melhor da geração  4 :  [-108, -76, 221]  com fitness  5
Melhor da geração  5 :  [-57, 27, 128]  com fitness  5


### Usando o algoritmo de coevolução para outros jogos - Exemplo do jogo do Galo (extra)

A modelização do jogo do Galo é copiada do guião da PL5, apenas com a adição dos atributos `first` e `second` à classe TicTacToe, tal como vem especificado no enunciado.

São usadas 6 funções características: o número de centros que jogador tem, o número de centros que oponente tem, o número de cantos que jogador tem, o número de cantos que oponente tem, o número de laterais ((1,2) é um exemplo de uma lateral) que jogador tem e o número de laterais que oponente tem.

Primeiro os imports de IIA2021-proj3-42:

In [80]:
import importlib
imported_p3 = importlib.import_module("IIA2021-proj3-42")

TicTacToe = imported_p3.TicTacToe

caract_num_centros_player = imported_p3.f_num_centros_player
caract_num_centros_oponent = imported_p3.f_num_centros_oponent
caract_num_cantos_player = imported_p3.f_num_cantos_player
caract_num_cantos_oponent = imported_p3.f_num_cantos_oponent
caract_num_laterais_player = imported_p3.f_num_laterais_player
caract_num_laterais_oponent = imported_p3.f_num_laterais_oponent

Os parâmetros:

In [81]:
gen = 10
dim = 16
gene_pool = [i for i in range(-250,251,1)]
elit = 0.2
p_mut = 0.25
k_rivais = 2
lim_profs = [1,2]
n_jogos = 5
jogo = TicTacToe
caracts = [caract_num_centros_player, caract_num_centros_oponent, 
           caract_num_cantos_player, caract_num_cantos_oponent, 
           caract_num_laterais_player, caract_num_laterais_oponent]

E a execução:

In [82]:
best_chromosomes = coevol(gen,dim,gene_pool,elit,p_mut,k_rivais,lim_profs,n_jogos,jogo,caracts)

for i, c in enumerate(best_chromosomes):
    print("Melhor da geração ", i, ": ", c[0], " com fitness ", c[1])

Melhor da geração  0 :  [-127, 81, 207, 10, -215, -55]  com fitness  5
Melhor da geração  1 :  [-173, 102, 58, 193, -215, -55]  com fitness  5
Melhor da geração  2 :  [-127, 81, 207, 10, -154, -75]  com fitness  5
Melhor da geração  3 :  [-173, 102, 58, 193, -215, -53]  com fitness  5
Melhor da geração  4 :  [-119, 86, 207, 7, -154, -75]  com fitness  5
Melhor da geração  5 :  [-173, 97, 58, 192, -215, -69]  com fitness  5
Melhor da geração  6 :  [-127, 74, 207, 10, -208, -75]  com fitness  5
Melhor da geração  7 :  [-174, 102, 58, 187, -217, -71]  com fitness  5
Melhor da geração  8 :  [-130, 72, 207, 13, -144, -87]  com fitness  5
Melhor da geração  9 :  [-127, 72, 207, 187, -195, -64]  com fitness  5
Melhor da geração  10 :  [-161, 104, 199, 19, -150, -68]  com fitness  5
